In [ ]:
from langchain_openai import ChatOpenAI 
from langgraph.graph import StateGraph, END, MessagesState 
from langgraph.prebuilt import ToolNode 
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_community.tools import TavilySearchResults 
from langchain_community.agent_toolkits.load_tools import load_tools
import os
import json

os.environ['TAVILY_API_KEY'] = "your_api_key"
os.environ['OPENAI_API_KEY'] = "your_api_key"

In [8]:
#Tavily do wyszukiwania w Internecie
search_tool = TavilySearchResults(max_results=10, 
                                  include_answer=True, 
                                  include_raw_content=False)
#Human tool dla interakcji z użytkownikiem
human_tool = load_tools(["human"])[0]

tools = [search_tool, human_tool]
#Chat-GPT 4o jako duży model językowy
llm = ChatOpenAI(model="gpt-4o", max_tokens=None).bind_tools(tools)

In [9]:
# Funkcja, która określa, czy należy wywołać narzędzia,
# czy wynik został już uzyskany.
def should_continue(state: MessagesState):
    messages = state["messages"]
    last_message = messages[-1]
    if last_message.tool_calls:
        return "tools"
    return END


In [10]:
def gather_data(state: MessagesState):

    messages = state["messages"]

    messages.append(SystemMessage(content='''
    Jesteś asystentem AI w pewnym banku odpowiedzialnym za zbieranie danych od użytkownika w celu dalszego przetwarzania zgłoszenia. 
    Klient zgłosił się z problemem. Twoim zadaniem jest zapamiętać wszystkie informacje podane przez użytkownika.

    Musisz zebrać i zapamiętać następujące dane:
    ** Imię i nazwisko
    ** poczta elektroniczna
    ** numer telefonu
    Każdą z tych informacji zachowaj w pamięci systemu i użyj ich później do wygenerowania kompletnej odpowiedzi w formacie JSON. 
    Jeśli którejkolwiek z danych brakuje, poproś użytkownika o jej uzupełnienie.

    Pamiętaj, aby nie zapomnieć wcześniej podanych informacji — nawet jeśli użytkownik poda je w kilku krokach.

    Do wyszukiwania informacji użyj narzędzia TavilySearchResults.
    Za pomocą HumanInputRun możesz dopytać klienta o jego dane oraz szczegóły dotyczące problemu.
      
    Odpowiedz w formacie JSON: 
    {{"case_data": wszystkie dane o kliencie zapisz formie tekstowej}}.
    W odpowiedzi nie mogą znajdować się żadne inne dane – 
    tylko poprawny, możliwy do przetworzenia JSON.
    '''))

    response = llm.invoke(messages)

    
    
    print(json.dumps(response.tool_calls, indent=2,ensure_ascii=False))
    print(json.dumps(response.content, indent=2,ensure_ascii=False))
   
    return {"messages": [response]}

In [11]:
tool_node = ToolNode(tools)
workflow = StateGraph(MessagesState)


workflow.add_node("gather_data_node", gather_data)
workflow.add_node("tools", tool_node)

workflow.set_entry_point("gather_data_node")
workflow.add_conditional_edges("gather_data_node", 
                               should_continue, 
                               ["tools", END])

workflow.add_edge("tools", "gather_data_node")
graph = workflow.compile()

In [12]:
prompt = """
    Oto opis sytuacji przedstawiony przez klienta:
    Chciałbym dowiedzieć się co to jest PIT?. Nazywam się Bogdan Kiewlak, adres email: example@email.com, numer telefonu: +123456789
    """
input_messages = [HumanMessage(prompt)]
output = graph.invoke({"messages": input_messages})

[
  {
    "name": "tavily_search_results_json",
    "args": {
      "query": "Co to jest PIT?"
    },
    "id": "call_1q287tAVT9QBsXMdYfXF2t6G",
    "type": "tool_call"
  }
]
""
[]
"{\"case_data\":\"Klient: Bogdan Kiewlak, Adres email: example@email.com, Numer telefonu: +123456789, Zapytanie: Chciałbym dowiedzieć się co to jest PIT? Informacja: Podatek dochodowy od osób fizycznych, znany jako PIT, to podatek bezpośredni obejmujący dochody uzyskiwane przez osoby fizyczne. Jest to zarówno nazwa samego podatku, jak i dokumentu urzędowego, który podatnicy wypełniają i składają do odpowiedniego urzędu skarbowego, aby rozliczyć swoje dochody roczne. Podatek ten dotyczy różnych źródeł przychodu, w tym wynagrodzenia za pracę czy przychody z działalności gospodarczej.\"}"
